# Datos entreneamiento Shift

En este notebook vamos a aplicar el shift para las obtener las columnas mas correlacionadas.

- __Datos recibidos:__ obtenidos de notebook __1-datos_entrenamiento_shift.ipynb__
- __Responsable:__ Daniel Bustillos
- __Contacto:__  juandaniel.bucam@gmail.com

__Notas del proyecto__
Necesitamos generar un csv con los niveles máximos, mínimo y promedio por día, se perderá la información de la estación, también generaremos el atributo PM10mean y PM25mean que representarán los valores de las últimas 24 horas de los contaminantes.

calidad de caire
pronostico contaminación

## Pasos
[X] Aplicar Shift con Correlaciones <br>
[X] Sacar minimo promedio y maximo <br>
[X] Hacer el mes y el día columnas<br><br>
[X] Hacer el rollinng para PM10 y PM25<br><br>

juntar<br>
[X] juntar archivos de append y rolling <br><br>


In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None 

In [2]:
data_hour_merge_24 = pd.read_csv('./datos_entrenamiento_10-19.csv')

Quedémonos sin WSR:

In [3]:
data_hour_merge_24 = data_hour_merge_24[['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', "PM2.5",'RH', 'SO2', 'TMP',
       'WSP', 'dia', 'fecha', 'hora', 'id_station', 'mes',"PM10mean","PM25mean"]].reset_index(drop=True)

Hagamos una lista con todas las estaciones:

### Arreglemos las fechas:

In [4]:
data_hour_merge_24 = data_hour_merge_24[data_hour_merge_24.fecha > "2016-01-01 23:00:00"]
data_hour_merge_24['dia'] = data_hour_merge_24['dia'].astype(str).astype(float)
data_hour_merge_24['mes'] = data_hour_merge_24['mes'].astype(str).astype(float)

Creemos un diccionario con __data_hour_merge_24__ dividido por estación:

In [5]:
estaciones = data_hour_merge_24.id_station.unique().tolist()

In [6]:
data_est = {}
for elem in estaciones:
    data_est[elem] = data_hour_merge_24[data_hour_merge_24.id_station == elem]

# Leamos el archivo de correlaciones del PM10 mean

Este archivo se generó en el notebook: __/home/paw/DanielBustillos/contaminación/correlaciones_pau/correlaciones_función_paulina.ipynb__

In [7]:
O3_corr = pd.read_csv('/home/paw/DanielBustillos/contaminación/correlaciones_pau/PM10_corr.csv')
O3_corr = O3_corr.iloc[:,1:]

Vamos a quedarnos con las variables con corr>0.3:

In [8]:
O3_corr_filtro = O3_corr[(O3_corr.valor>0.4) | (O3_corr.valor>0.4) ].reset_index(drop=True)

Vamos a aplicar el shift para cada elemento de la tabla __O3_corr__:

In [9]:
def shit_corr(df):
    for i in range(len(O3_corr_filtro)):
        name_column = str( O3_corr_filtro.loc[i,"contaminante"] + "_" + str( O3_corr_filtro.loc[i,"horas"] ) ) 
        df[name_column] = df[O3_corr_filtro.loc[i,"contaminante"]].shift( - int(str(O3_corr_filtro.loc[i,"horas"])) ) 
    return df.dropna()

Aplicamos esta función para cada DF de cada estación:

In [10]:
data_shift = {}
for elem in data_est:
    data_shift[elem] = shit_corr(data_est[elem])

In [11]:
df_append = pd.DataFrame(columns = data_est["MER"].columns.tolist())

In [12]:
for key in data_est:
    df_append = df_append.append(data_est[key], ignore_index=True,sort=True)

## Groupbys

In [13]:
cols = df_append.columns.tolist()

In [14]:
cols.remove("id_station")

Finalmente, aplicamos los groupbys:

In [15]:
data_hour_merge_24_mean = df_append.groupby(['fecha',"hora","dia","mes"]).mean()
data_hour_merge_24_mean.reset_index(inplace=True)

In [16]:
data_hour_merge_24_max = df_append.groupby(['fecha',"hora","dia","mes"]).max()
data_hour_merge_24_max.reset_index(inplace=True)

In [17]:
data_hour_merge_24_min = df_append.groupby(['fecha',"hora","dia","mes"]).min()
data_hour_merge_24_min.reset_index(inplace=True)

##### Unamos los df's de manera que tengamos en una solo los datos promedio, máximo y minimo por día:

In [23]:
data_hour_merge = pd.merge(data_hour_merge_24_mean, data_hour_merge_24_max, on=['fecha',"hora","dia","mes"])
data_hour_merge = pd.merge(data_hour_merge, data_hour_merge_24_min, on=['fecha',"hora","dia","mes"])

guardamos el archivo:

## Leamos el archivo:

In [124]:
data_hour_merge = pd.read_csv("./datos_gb_PM10.csv", sep=',', encoding='utf-8')

In [125]:
data_hour_merge.head(5)

,fecha,hora,dia,mes,CO_x,NO_x,NO2_x,NO2_0_x,NOX_x,O3_x,...,PM25mean_5,PM25mean_6,PM25mean_7,PM25mean_8,PM25mean_9,RH,SO2,TMP,WSP,id_station_y
0,2016-01-02 00:00:00,0,2.0,1.0,0.950000,22.666667,40.000000,40.000000,62.666667,2.500000,...,25.416667,26.666667,27.791667,28.708333,29.416667,55.0,9.0,15.9,1.3,HGM
1,2016-01-02 01:00:00,1,2.0,1.0,0.500000,3.833333,21.166667,21.166667,25.166667,8.500000,...,21.041667,21.250000,21.166667,21.000000,20.125000,73.0,6.0,12.0,1.4,HGM
2,2016-01-02 02:00:00,2,2.0,1.0,0.500000,4.166667,20.166667,20.166667,24.333333,7.833333,...,21.125000,21.041667,21.250000,21.166667,21.000000,74.0,4.0,12.1,1.2,HGM
3,2016-01-02 03:00:00,3,2.0,1.0,0.466667,3.500000,18.166667,18.166667,21.333333,8.000000,...,21.041667,21.125000,21.041667,21.250000,21.166667,73.0,2.0,12.0,1.2,HGM
4,2016-01-02 04:00:00,4,2.0,1.0,0.500000,2.833333,17.333333,17.333333,20.333333,6.000000,...,21.000000,21.041667,21.125000,21.041667,21.250000,73.0,2.0,11.9,1.1,HGM


## Correlacionados con el target.

Ahora vamos a sacar los atributos más correlacionado con el target, el target es la columna a pronosticar, por simplicidad solo vamos a sacar la correlación con target a las 12 horas:

Generemos los target de pronóstico:

Generemos el shift de 1 a 24 horas, ahora hacia adelante

In [126]:
item = "PM10mean_y"
for i in range(1, 25):
    col_name = str(item+"_frcst_"+str(i))
    data_hour_merge[col_name] = data_hour_merge[item].shift(i)
data_hour_merge.dropna(inplace=True)

Saquemos la correlación:

In [127]:
data_corr = data_hour_merge.corr()

Guardamos los atributos mas correlacionados en una lista:

In [128]:
variables_mas_correlacionadas_pm10_mean = data_corr["PM10mean_y_frcst_24"][(data_corr["PM10mean_y_frcst_24"] > 0.2) |
                                                                           (data_corr["PM10mean_y_frcst_24"] < -0.2)].index.tolist()

In [129]:
variables_mas_correlacionadas_pm10_mean.append("TMP")
variables_mas_correlacionadas_pm10_mean.append("WSP")

Guardemos estas variables, no pueden cambiarse una vez que los modelos estan entrenados:

In [138]:
with open("./variables+correlacionadas/vars_mas_corr_pm10mean.txt", "w") as output:
    output.write(str(variables_mas_correlacionadas_pm10_mean))

Filtremos el DF con las variables mas correlacionadas:

In [131]:
data_hour_merge = data_hour_merge[variables_mas_correlacionadas_pm10_mean]

Añadamos los datos de pronostico de las siguientes variables:

In [132]:
lista_frcst = ["TMP","WSP","RH"] 

In [133]:
for item in lista_frcst:
    for i in range(1, 25):
        col_name = str(item+"_frcst_"+str(i))
        data_hour_merge[col_name] = data_hour_merge[item].shift(i)
data_hour_merge.dropna(inplace=True)

Guardemos: